In [1]:
# Import dependencies
import os

os.environ['TF_CPP_MIN_VLOG_LEVEL'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

from src.utils.gpu_config import configure_gpu, optimize_tensorflow

configure_gpu()
optimize_tensorflow()

2025-06-19 08:52:22.806461: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750315942.885853    3449 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750315942.909566    3449 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750315943.098728    3449 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750315943.098736    3449 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750315943.098738    3449 computation_placer.cc:177] computation placer alr


GPU configuration successful!
Device: /physical_device:GPU:0
Memory limit: 18.0 GB

TensorFlow optimizations status:
- CPU Threads: 16 intra-op, 9 inter-op (32 cores total)
- XLA JIT: enabled (autoclustering)
- Synchronous Execution: enabled

GPU Device Details:
- Device: GPU /physical_device:GPU:0
- Memory limit: 18.0 GB


I0000 00:00:1750315946.397743    3449 gpu_process_state.cc:208] Using CUDA malloc Async allocator for GPU: 0
I0000 00:00:1750315946.398515    3449 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 19327352832 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6


In [2]:
# Import Tensorflow Keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryFocalCrossentropy
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

# Import local modules
from src.utils.consts import TF_RECORD_DATASET, MODELS_PATH, TF_BUFFER_SIZE, TF_SHUFFLE_SIZE, TF_BATCH_SIZE, TF_MAX_EPOCHS
from src.model.tensorflow_utils import load_dataset, optimize_dataset, count_dataset_size, get_num_classes, get_metrics
from src.model.tensorflow_utils import setup_logger, setup_training_logger, setup_metrics_monitor, setup_early_stopping, filter_no_finding_class
from src.model.tensorflow_utils import start_or_resume_training, apply_augmentation_to_dataset, calculate_class_weights, oversample_minority_classes
from src.model.densnet.tensorflow_dense_net_121 import build_densenet121

# Input Data
initial_epoch   = 83
resume_training = True
checkpoint_path = '/home/piotr/Pulpit/codebook/studies/bachelor-thesis/models/DenseNetV6_1/checkpoints/cp-0082.keras'
set_name        = "DenseNetV6"
model_name      = f"{set_name}_1"
num_classes     = get_num_classes()

train_ds = load_dataset(f"{TF_RECORD_DATASET}/train.tfrecord", TF_BUFFER_SIZE)
val_ds   = load_dataset(f"{TF_RECORD_DATASET}/val.tfrecord", TF_BUFFER_SIZE)

train_ds = filter_no_finding_class(train_ds)
val_ds   = filter_no_finding_class(val_ds)

class_weights = calculate_class_weights(train_ds, num_classes)
train_ds      = oversample_minority_classes(train_ds, class_weights)
class_weights = calculate_class_weights(train_ds, num_classes)

In [3]:
steps_per_epoch  = int(count_dataset_size(train_ds, None) / TF_BATCH_SIZE)
validation_steps = int(count_dataset_size(val_ds, None) / TF_BATCH_SIZE)

# Disable Advanced Augmentations
probability = {
    "intensity_scaling": 0,
    "adaptive_histogram": 0,
    "zoom": 0,
    "cutout": 0
}

train_ds = train_ds.shuffle(TF_SHUFFLE_SIZE, reshuffle_each_iteration=True)
train_ds = apply_augmentation_to_dataset(train_ds, probability)
train_ds = optimize_dataset(train_ds, TF_BATCH_SIZE)

val_ds   = optimize_dataset(val_ds, TF_BATCH_SIZE)

Counting samples: 104166 samples [00:36, 2866.33 samples/s] 
Counting samples: 15431 samples [00:09, 1695.75 samples/s]


In [4]:
# Setup Model Deps
logger            = setup_logger()
training_logger   = setup_training_logger(logger, TF_BATCH_SIZE, 100)
metrics_monitor   = setup_metrics_monitor(MODELS_PATH, model_name, logger, resume_training=resume_training, initial_epoch=initial_epoch)
metrics           = get_metrics()

# Setup compile arguments
loss           = BinaryFocalCrossentropy(gamma=2.0, from_logits=False, label_smoothing=0.05)
reduce_lr      = ReduceLROnPlateau(monitor="val_f1_score", factor=0.5,  patience=3, min_lr=1e-6, mode="max", verbose=1)
early_stopping = setup_early_stopping()

epoch_mode           = 'cp-{epoch:04d}'
save_checkpoint_path = f"{MODELS_PATH}/{model_name}/checkpoints/{epoch_mode}.keras"
checkpoint           = ModelCheckpoint(save_checkpoint_path, monitor="val_f1_score", save_best_only=False, mode="max", save_weights_only=False)

model_path      = f"{MODELS_PATH}/{model_name}.keras"
best_checkpoint = ModelCheckpoint(model_path, monitor="val_f1_score", save_best_only=True, mode="max", save_weights_only=False)

2025-06-19 08:54:30 - INFO - Resuming from existing metrics file: /home/piotr/Pulpit/codebook/studies/bachelor-thesis/models/DenseNetV6_1/train_metrics.csv
2025-06-19 08:54:30 - INFO - Resuming from existing validation metrics file: /home/piotr/Pulpit/codebook/studies/bachelor-thesis/models/DenseNetV6_1/val_metrics.csv
2025-06-19 08:54:30 - INFO - Cleaned training metrics file, kept 66666 records before epoch 83
2025-06-19 08:54:30 - INFO - Cleaned validation metrics file, kept 82 records before epoch 83
2025-06-19 08:54:30 - INFO - Found 66666 existing training records
2025-06-19 08:54:30 - INFO - Found 82 existing validation records


In [5]:
# Model Training
model          = build_densenet121(num_classes, use_se=True, layer_attention=False, ratio=16, channel_blocks=[0, 1, 2, 3])
optimizer      = Adam(learning_rate=1e-4)
compile_kwargs = {'optimizer': optimizer, 'loss': loss, 'metrics': metrics}

history, model = start_or_resume_training(
    model, 
    compile_kwargs, 
    train_ds, 
    val_ds, 
    TF_MAX_EPOCHS,
    steps_per_epoch, 
    validation_steps, 
    class_weights=class_weights,
    callbacks=[checkpoint, best_checkpoint, reduce_lr, training_logger, metrics_monitor, early_stopping], 
    checkpoint_path=checkpoint_path,
    initial_epoch=initial_epoch,
    output_dir=MODELS_PATH,
    model_name=model_name,
    logger=logger
)

2025-06-19 08:54:31 - INFO - Resuming training from checkpoint: /home/piotr/Pulpit/codebook/studies/bachelor-thesis/models/DenseNetV6_1/checkpoints/cp-0082.keras
2025-06-19 08:54:33 - INFO - Resuming from existing metrics file: /home/piotr/Pulpit/codebook/studies/bachelor-thesis/models/DenseNetV6_1/train_metrics.csv
2025-06-19 08:54:33 - INFO - Resuming from existing validation metrics file: /home/piotr/Pulpit/codebook/studies/bachelor-thesis/models/DenseNetV6_1/val_metrics.csv
2025-06-19 08:54:33 - INFO - Cleaned training metrics file, kept 66666 records before epoch 83
2025-06-19 08:54:33 - INFO - Cleaned validation metrics file, kept 82 records before epoch 83
2025-06-19 08:54:33 - INFO - Found 66666 existing training records
2025-06-19 08:54:33 - INFO - Found 82 existing validation records
2025-06-19 08:54:33 - INFO - 
=== Training Started ===

2025-06-19 08:54:33 - INFO - Batch Size: 128
2025-06-19 08:54:33 - INFO - Optimizer: Adam
2025-06-19 08:54:33 - INFO - 

2025-06-19 08:54:3

Epoch 83/100


I0000 00:00:1750316093.133774    4106 service.cc:152] XLA service 0x7a0c30047e40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1750316093.133847    4106 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3090 Ti, Compute Capability 8.6
I0000 00:00:1750316098.516433    4106 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1750316162.386925    4106 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


813/813 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step - accuracy: 0.9565 - auc: 0.9629 - f1_score: 0.6667 - loss: 0.0232 - precision: 0.7960 - recall: 0.5944  

2025-06-19 08:59:18 - INFO - 
=== Epoch 83 Summary ===
2025-06-19 08:59:18 - INFO - Time: 285.20s
2025-06-19 08:59:18 - INFO - Training   - accuracy: 0.9567 - auc: 0.9633 - f1_score: 0.6678 - loss: 0.0231 - precision: 0.7964 - recall: 0.5963 - learning_rate: 0.0000
2025-06-19 08:59:18 - INFO - Validation - accuracy: 0.9323 - auc: 0.7504 - f1_score: 0.2372 - loss: 0.0787 - precision: 0.3190 - recall: 0.1956
2025-06-19 08:59:18 - INFO - ==================================================



...

Epoch 100/100
813/813 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step - accuracy: 0.9599 - auc: 0.9685 - f1_score: 0.6981 - loss: 0.0219 - precision: 0.8154 - recall: 0.6288  

2025-06-19 09:55:31 - INFO - 
=== Epoch 100 Summary ===
2025-06-19 09:55:31 - INFO - Time: 194.23s
2025-06-19 09:55:31 - INFO - Training   - accuracy: 0.9599 - auc: 0.9686 - f1_score: 0.6974 - loss: 0.0219 - precision: 0.8136 - recall: 0.6288 - learning_rate: 0.0000
2025-06-19 09:55:31 - INFO - Validation - accuracy: 0.9309 - auc: 0.7486 - f1_score: 0.2529 - loss: 0.0841 - precision: 0.3181 - recall: 0.2162
2025-06-19 09:55:31 - INFO - ==================================================



813/813 ━━━━━━━━━━━━━━━━━━━━ 194s 239ms/step - accuracy: 0.9599 - auc: 0.9685 - f1_score: 0.6981 - loss: 0.0219 - precision: 0.8154 - recall: 0.6288 - val_accuracy: 0.9309 - val_auc: 0.7486 - val_f1_score: 0.2529 - val_loss: 0.0841 - val_precision: 0.3181 - val_recall: 0.2162 - learning_rate: 1.0000e-06


2025-06-19 09:55:31 - INFO - 
=== Training Completed! ===

2025-06-19 09:55:31 - INFO - Final Metrics: accuracy: 0.9599 - auc: 0.9686 - f1_score: 0.6974 - loss: 0.0219 - precision: 0.8136 - recall: 0.6288 - val_accuracy: 0.9309 - val_auc: 0.7486 - val_f1_score: 0.2529 - val_loss: 0.0841 - val_precision: 0.3181 - val_recall: 0.2162



Restoring model weights from the end of the best epoch: 99.
